In [10]:
from hyper import *
from alignment import *
from datasets.preprecossing import *
from core import *
from datasets.loading import *
from datasets.hc_dataset import *
from datasets.balance_dataset import *
from utils.linkage import *
from model.balancehc import balancehc

from utils.poincare import *


In [11]:
cell_path1 = './datas/23/2/BRCA_GSE110686.h5ad'
folder_path1 = './datas/23/2/'
radius1 = 0
c1 =0
epoches1 =20
cell_path2 = "./datas/23/3/CRC_GSE146771.h5ad" 
folder_path2 = "./datas/23/3/" 
radius2 = 0
c2 =0
epoches2 = 20
contin = False
resolution=1
method='average'
alignment=1
n_pca=50
meta_col = 'Celltype..major.lineage.'
mst = True


In [3]:
    get_Hyper_tree(folder_path1+'datas.data',1,tmp1.shape[1]+1,0,epoches1,epoches1,save_path=folder_path1,mst=mst)


NameError: name 'tmp1' is not defined

In [13]:
np.random.seed(1234)
torch.manual_seed(1234)
x, y_true, similarities = load_data(folder_path1+'datas.data',1,51,0)
print("{} length:{}".format(folder_path1+'datas.data',len(y_true)));
dataset = HCDataset(x, y_true, similarities, num_samples=50000)
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
model = HypHC(dataset.n_nodes, 2, 5e-2, 5e-2 ,0.999)

if(None==None or os.path.exists('model_path')==False):
    model.to("cpu")
    Optimizer = getattr(optim, 'RAdam')
    optimizer = Optimizer(model.parameters(),0.0005)
    train(model,dataloader,optimizer,similarities,epoches1);
    # torch.save(model.state_dict(),save_path+'model.pth');
else:
    # params = torch.load((model_path), map_location=torch.device('cpu'))
    model.load_state_dict(params, strict=False)

model.eval()

sim_fn = lambda x, y: torch.sum(x * y, dim=-1)
n=len(x);
leaves_embeddings = model.normalize_embeddings(model.embeddings.weight.data)
leaves_embeddings = project(leaves_embeddings).detach().cpu()
ijs = sl_np_mst_ij(leaves_embeddings,sim_fn)
uf = UnionFind(n)
uf.merge(ijs)

tree = nx.DiGraph()
for i, j in enumerate(uf.tree()[:-1]):
    if(j!=-1):
        tree.add_edge(j, i)

# fig = plt.figure(figsize=(15, 15))
# ax = fig.add_subplot(111)
# circle = plt.Circle((0, 0), 20.0, color='r', alpha=0.1)
# ax.add_artist(circle)

n = len(leaves_embeddings)
embeddings = complete_tree(tree, leaves_embeddings)

./datas/23/2/datas.data length:10
Generating all pairs superset
	 Epoch 0 | average train loss: 0.966356
Optimization finished.
	 Epoch 1 | average train loss: 0.941207
Optimization finished.
	 Epoch 2 | average train loss: 0.941279
Optimization finished.
	 Epoch 3 | average train loss: 0.940414
Optimization finished.
	 Epoch 4 | average train loss: 0.940350
Optimization finished.
	 Epoch 5 | average train loss: 0.940468
Optimization finished.
	 Epoch 6 | average train loss: 0.941214
Optimization finished.
	 Epoch 7 | average train loss: 0.940824
Optimization finished.
	 Epoch 8 | average train loss: 0.940168
Optimization finished.
	 Epoch 9 | average train loss: 0.939446
Optimization finished.
	 Epoch 10 | average train loss: 0.939698
Optimization finished.
	 Epoch 11 | average train loss: 0.938994
Optimization finished.
	 Epoch 12 | average train loss: 0.940198
Optimization finished.
	 Epoch 13 | average train loss: 0.940589
Optimization finished.
	 Epoch 14 | average train loss: 0.9

In [15]:
nodes1 = [node(name=str(i),son=[]) for i in range(len(uf.tree()))]
for i in range(n):
    nodes1[i].subson=[i];
for i,j in enumerate(uf.tree()):
    if(j!=-1):
        nodes1[j].son.append(nodes1[i])
    nodes1[i].value=torch.tensor(embeddings[i]);
    nodes1[j].subson.extend(nodes1[i].subson)
root = nodes1[-1];

values = [];
fathers = [];
xys = [];
search_merge_tree(root,-1,0,values,fathers,xys)

In [20]:
root

18

In [21]:
deep_search_tree(root,0,[],root)
result = []
distances = []
for index in range(n,len(nodes1)):
    i = nodes1[index]
    if(len(i.subson)==2):
        for j in i.rest(n):
            result.append([i.subson,j,int(i),1,int(j)])
            
for index in range(n,len(nodes1)-1):
    i = nodes1[index]
    for i1 in range(len(i.subson)):
        for i2 in range(i1+1,len(i.subson)):
            for j in i.rest(n):
                result.append([[i.subson[i1],i.subson[i2]],j,int(i),0,int(i.f)])
        
for i in nodes1:
    distances.append(i.distance_to_root);

In [31]:
result[0]

[[0, 6], 1, 10, 1, 1]

In [ ]:
python run_sc.py -cp1 './datas/120/3/sample1_small.h5' -f1 "./datas/120/3/" -r1 25 -c1 0.1 -e1 10 -cp2 './datas/120/4/sample1_small.h5' -f2 "./datas/120/4/" -r2 25 -c2 0.1 -e2 10 --contin False --alignment 1 --resolution 1 --n_pca 500


In [4]:
if (contin==False) or ((os.path.exists(folder_path1+'merge_cell_data.csv') and os.path.exists(folder_path1 + 'merge_cell_meta.csv')) == False):
    loss1 = merge_by_radius(cell_path1,folder_path1,radius1,method,meta_col)
    print("cell meta score for dataset1: {}\n".format(loss1))
else:
    print("dataset1 find files and skip merging")


adata1 = sc.read_h5ad(folder_path1+ 'adata.h5ad')

if(contin==False) or ((os.path.exists(folder_path2+'merge_cell_data.csv') and os.path.exists(folder_path2 + 'merge_cell_meta.csv')) == False):
    loss2 = merge_by_radius(cell_path2,folder_path2,radius2,method,meta_col)
    print("cell meta score for dataset2: {}".format(loss2))
else:
    print("dataset2 find files and skip merging")

adata2 = sc.read_h5ad(folder_path2+ 'adata.h5ad')


preprocessing_cluster(adata1,N_pcs=n_pca,resolution=resolution)
preprocessing_cluster(adata2,N_pcs=n_pca,resolution=resolution)

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1) 
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)


meta_list1 = calculate_cluster_celltype(adata1);
meta_list2 = calculate_cluster_celltype(adata2);
calculate_meta_ori(folder_path1,adata1);
calculate_meta_ori(folder_path2,adata2);

if(contin==False) or ((os.path.exists(folder_path1 + 'dataxy.npy') and os.path.exists(folder_path1+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path1+'datas.data',1,tmp1.shape[1]+1,0,epoches1,epoches1,save_path=folder_path1,mst=mst)
else:
    print("dataset1 find files and skip embedding");

if(contin==False) or ((os.path.exists(folder_path2 + 'dataxy.npy') and os.path.exists(folder_path2+'data1link.npy') and os.path.exists(folder_path1+'dataname.npy')) == False):
    get_Hyper_tree(folder_path2+'datas.data',1,tmp2.shape[1]+1,0,epoches2,epoches2,save_path=folder_path2,mst=mst)
else:
    print("dataset2 find files and skip embedding")

    
nodes1 = build_hyper_tree_from_folder(folder_path1,mst)
nodes2 = build_hyper_tree_from_folder(folder_path2,mst)

merge_list1 = [];
merge_list2 = [];
nodes1[0] = search_tree(nodes1[0],c1,merge_list1)
nodes2[0] = search_tree(nodes2[0],c2,merge_list2)

for i in range(len(nodes1)):
    if(int(nodes1[i].name)<len(meta_list1)):
        nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
        
for i in range(len(nodes2)):
    if(int(nodes2[i].name)<len(meta_list2)):
        nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  
        
rate = 0;        
if(alignment==1):
    rate,anslist,ans = run_alignment(nodes1,nodes2,folder_path1,folder_path2,meta_list1,meta_list2);
elif(alignment==2):
    rate = run_alignment_linear(nodes1,nodes2);

dataset1 find files and skip merging


/var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/ipykernel_96918/2407986756.py:12: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata1 = anndata.AnnData(adata1)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


dataset2 find files and skip merging


/var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/ipykernel_96918/2407986756.py:26: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata2 = anndata.AnnData(adata2)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


100 2163


...

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/datasets/balance_dataset.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp = np.array(temp)
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/hyper.py:346: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model2 = balancehc(nodes1,torch.tensor(embeddings),hyperparamter = 1)
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTen

	 Epoch 0 | average train loss: 0.366166


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[i]),torch.tensor(self.original[j]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[int(nodef.son[0])]),torch.tensor(self.original[k])) + hyp_dist(torch.tensor(self.original[int(nodef.son[1])]),torch.tensor(self.original[k]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:92: UserWarning: To copy construct from a tensor, it is recommended 

	 Epoch 1 | average train loss: 0.366592
	 Epoch 2 | average train loss: 0.367283
	 Epoch 3 | average train loss: 0.366894
	 Epoch 4 | average train loss: 0.363955
./datas/120/4/datas.data length:3
Generating all pairs superset
	 Epoch 0 | average train loss: 0.745610
Optimization finished.
	 Epoch 1 | average train loss: 0.702685
Optimization finished.
	 Epoch 2 | average train loss: 0.713906
Optimization finished.
	 Epoch 3 | average train loss: 0.687782
Optimization finished.
	 Epoch 4 | average train loss: 0.691657
Optimization finished.
	 Epoch 0 | average train loss: 0.188568
	 Epoch 1 | average train loss: 0.176657
	 Epoch 2 | average train loss: 0.185171
	 Epoch 3 | average train loss: 0.179024
	 Epoch 4 | average train loss: 0.184698
The mininum cost for alignment is 4.211409806398651
The alignment edges list is [('8', '4'), ('6', '3'), ('2_CD8T', '1_CD8T'), ('0_CD8T', '0_CD8T'), ('1_CD4Tconv', '2_CD4Tconv')]
average cost for one node:0.3008149861713322

correct alignment rate

1.0

In [7]:
ori1 = sc.read_h5ad(cell_path1)
ori1

AnnData object with n_obs × n_vars = 5000 × 16701
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'Cluster', 'Celltype..malignancy.', 'Celltype..major.lineage.', 'Celltype..minor.lineage.', 'Patient', 'Sample', 'Tissue', 'celltype'
    var: 'features'

In [8]:
ori2 = sc.read_h5ad(cell_path2)
ori2

AnnData object with n_obs × n_vars = 5000 × 16701
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'Cluster', 'Celltype..malignancy.', 'Celltype..major.lineage.', 'Celltype..minor.lineage.', 'Patient', 'Sample', 'Tissue', 'celltype'
    var: 'features'

In [10]:
ori1.obs

,orig.ident,nCount_RNA,nFeature_RNA,UMAP_1,UMAP_2,Cluster,Celltype..malignancy.,Celltype..major.lineage.,Celltype..minor.lineage.,Patient,Sample,Tissue,celltype
Tcells_PP@TTCTACAGTCACTTCC,1,2641.725803,1555,-5.280918,5.104487,8,Immune cells,CD8T,CD8Teff,Tcells_PP,Tcells_PP,Tumor,CD8T
CART_RP@AAATGCCGTTTGCATG,0,2162.938686,1179,2.772011,-6.456879,1,Immune cells,CD8T,CD8Teff,CART_RP,CART_RP,Tumor,CD8T
Tcells_RP@AACACGTCACCGCTAG,1,1715.759559,795,1.092078,-3.652448,6,Immune cells,CD8T,CD8Teff,Tcells_RP,Tcells_RP,Tumor,CD8T
CART_RP@CTGTTTAGTAAGAGAG,0,1809.857327,916,0.734953,-6.490685,1,Immune cells,CD8T,CD8Teff,CART_RP,CART_RP,Tumor,CD8T
CART_PP@GACAGAGTCCGAATGT,0,3195.854236,3425,-3.039640,10.285129,15,Immune cells,CD8T,CD8Teff,CART_PP,CART_PP,Tumor,CD8T
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CART_PP@ATGAGGGAGTTCGCAT,0,2469.901828,1489,-5.185951,5.364392,8,Immune cells,CD8T,CD8Teff,CART_PP,CART_PP,Tumor,CD8T
Tcells_RP@GAGTCCGAGGATGCGT,1,1727.467654,870,-3.970054,-5.204175,3,Immune cells,CD4Tconv,Th17,Tcells_RP,Tcells_RP,Tumor,CD4Tconv
Tcells_RP@CCCAGTTAGCCCTAAT,1,1842.613875,901,1.804658,-5.202508,0,Immune cells,CD8T,CD8Teff,Tcells_RP,Tcells_RP,Tumor,CD8T
Tcells_PP@GATGAAAGTCAGCTAT,1,2547.830697,1687,-2.821990,2.794468,9,Immune cells,CD8Tex,CD8Tex,Tcells_PP,Tcells_PP,Tumor,CD8Tex


In [9]:
meta = pd.read_csv('./datas/120/3/meta_result.csv')
meta

,Unnamed: 0,0,label,first,second
0,0,CD8T,0,CD8Tex,2
1,1,CD8T,0,CD8Tex,2
2,2,CD8T,0,CD8Tex,2
3,3,CD8T,0,CD8Tex,2
4,4,CD8T,0,CD8Tex,2
...,...,...,...,...,...
4995,4995,CD8T,1979,CD8T,2
4996,4996,CD8T,1980,CD8T,0
4997,4997,CD8T,1981,CD8T,1
4998,4998,CD8T,1982,CD8T,0


In [6]:
adata2

AnnData object with n_obs × n_vars = 1547 × 2024
    obs: 'celltype', 'leiden'
    uns: 'pca', 'neighbors', 'diffmap_evals', 'umap', 'leiden', 'paga', 'leiden_sizes'
    obsm: 'X_pca', 'X_diffmap', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [5]:
adata1

AnnData object with n_obs × n_vars = 1984 × 2163
    obs: 'celltype', 'leiden'
    uns: 'pca', 'neighbors', 'diffmap_evals', 'umap', 'leiden', 'paga', 'leiden_sizes'
    obsm: 'X_pca', 'X_diffmap', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [15]:
T=tree_alignment(nodes1[0],nodes2[0],1);
minn = T.run_alignment();
T.show_ans();
ans = T.get_ans()
G=show_graph(ans,nodes1[0],nodes2[0])

The mininum cost for alignment is 4.460909902554654
The alignment edges list is [('12', '12'), ('11', '11'), ('9', '10'), ('6_CD4Tconv', '5_CD4Tconv'), ('2_CD4Tconv', '2_CD4Tconv'), ('0_CD8T', '0_CD8T'), ('8', '8'), ('7', '7'), ('5_CD8Tex', '3_CD8T'), ('4_CD8T', '1_CD8T'), ('1_CD8T', '6_CD8T')]


In [16]:
ans

[('12', '12'),
 ('11', '11'),
 ('9', '10'),
 ('6_CD4Tconv', '5_CD4Tconv'),
 ('2_CD4Tconv', '2_CD4Tconv'),
 ('0_CD8T', '0_CD8T'),
 ('8', '8'),
 ('7', '7'),
 ('5_CD8Tex', '3_CD8T'),
 ('4_CD8T', '1_CD8T'),
 ('1_CD8T', '6_CD8T')]

In [17]:
anslist_dist = dict(ans)
anslist_dist.keys()
def search_lineage(now,path,anss):
    path.append(now.name)
    if(now.son==[]):
        anss.append(path);
        return
    
    for i in now.son:
        search_lineage(i,path.copy(),anss);
temp1 = []
search_lineage(nodes1[0],[],temp1)
temp1

[['12', '8', '1_CD8T'],
 ['12', '8', '7', '4_CD8T'],
 ['12', '8', '7', '5_CD8Tex'],
 ['12', '11', '0_CD8T'],
 ['12', '11', '10', '3_CD4Tconv'],
 ['12', '11', '10', '9', '2_CD4Tconv'],
 ['12', '11', '10', '9', '6_CD4Tconv']]

In [18]:
anslist_dist

{'12': '12',
 '11': '11',
 '9': '10',
 '6_CD4Tconv': '5_CD4Tconv',
 '2_CD4Tconv': '2_CD4Tconv',
 '0_CD8T': '0_CD8T',
 '8': '8',
 '7': '7',
 '5_CD8Tex': '3_CD8T',
 '4_CD8T': '1_CD8T',
 '1_CD8T': '6_CD8T'}

In [19]:
route1 = []
route2 = []

for i in temp1:
    r1 = []
    r2 = []
    for j in i:
        if j in anslist_dist.keys():
            r1.append(j)
            r2.append(anslist_dist[j])
    route1.append(r1)
    route2.append(r2)
route2,route1


([['12', '8', '6_CD8T'],
  ['12', '8', '7', '1_CD8T'],
  ['12', '8', '7', '3_CD8T'],
  ['12', '11', '0_CD8T'],
  ['12', '11'],
  ['12', '11', '10', '2_CD4Tconv'],
  ['12', '11', '10', '5_CD4Tconv']],
 [['12', '8', '1_CD8T'],
  ['12', '8', '7', '4_CD8T'],
  ['12', '8', '7', '5_CD8Tex'],
  ['12', '11', '0_CD8T'],
  ['12', '11'],
  ['12', '11', '9', '2_CD4Tconv'],
  ['12', '11', '9', '6_CD4Tconv']])

In [20]:
show_the_tree(folder_path1)

In [21]:
route1

[['12', '8', '1_CD8T'],
 ['12', '8', '7', '4_CD8T'],
 ['12', '8', '7', '5_CD8Tex'],
 ['12', '11', '0_CD8T'],
 ['12', '11'],
 ['12', '11', '9', '2_CD4Tconv'],
 ['12', '11', '9', '6_CD4Tconv']]

In [22]:
show_the_tree(folder_path2)

In [25]:
adata1.obs['cluster'] = [i.name for i in nodes1]
adata1.obs

,cluster,dpt_pseudotime
0,12,0.000000
1,8,0.484134
2,1_CD8T,0.351462
3,7,0.593160
4,4_CD8T,0.445189
5,5_CD8Tex,0.448726
6,11,0.669826
7,0_CD8T,0.413974
8,10,0.776318
9,3_CD4Tconv,0.545269


In [26]:
adata1.X.mean(axis=0)

array([4.97558154e-03, 1.91218686e+00, 5.32756805e-01, 1.34270394e+00,
       1.90427661e+00, 9.85828280e-01, 8.17756262e-03, 6.82527572e-03,
       1.30788937e-01, 2.86819384e-04, 1.38684297e+00, 1.23750091e+00,
       2.76699364e-02, 5.58766365e-01, 1.95800085e-02, 1.49019372e+00,
       1.76119898e-02, 5.61720550e-01, 2.95884386e-02, 4.95642377e-03,
       1.78397369e+00, 1.33042455e+00, 3.16735059e-02, 9.32600424e-02,
       5.59931174e-02, 2.19713688e-01, 1.92287058e-01, 7.37559274e-02,
       9.20618400e-02, 2.50943433e-02, 5.71054989e-04, 3.83690349e-03,
       2.06112629e-03, 2.54351627e-02, 1.34101498e+00, 3.50452513e-02,
       1.97483916e-02, 2.22080224e-03, 5.63521087e-01, 2.39541242e-03,
       2.04178262e+00, 1.12984431e+00, 6.56114995e-01, 3.28297585e-01,
       1.13829947e+00, 1.12031960e+00, 5.74269518e-02, 5.22707961e-03,
       6.87986791e-01, 4.58287179e-01, 1.27944732e+00, 1.28603792e+00,
       1.19943060e-01, 5.97069832e-03, 9.35334623e-01, 1.47757316e+00,
      

In [24]:
data1 = []
for i in nodes1:
    data1.append(i.value)
data1 = np.array(data1)
adata1 = sc.AnnData(data1,dtype=np.float32)
adata1.obs['cluster'] = [i.name for i in nodes1]
sc.pp.neighbors(adata1,use_rep='X',n_neighbors=5)
sc.tl.diffmap(adata1)
adata1.var['xroot'] = adata1.X.mean(axis=0)
# adata1.uns['iroot'] = 0
# Compute diffusion pseudotime
sc.tl.dpt(adata1)
adata1

AnnData object with n_obs × n_vars = 13 × 974
    obs: 'cluster', 'dpt_pseudotime'
    var: 'xroot'
    uns: 'neighbors', 'diffmap_evals', 'iroot'
    obsm: 'X_diffmap'
    obsp: 'distances', 'connectivities'

In [27]:
data2 = []
for i in nodes2:
    data2.append(i.value)
data2 = np.array(data2)
adata2 = sc.AnnData(data2,dtype=np.float32)
adata2.obs['cluster'] = [i.name for i in nodes2]
sc.pp.neighbors(adata2,use_rep='X',n_neighbors=5)
sc.tl.diffmap(adata2)
adata2.var['xroot'] = adata2.X.mean(axis=0)
# adata1.uns['iroot'] = 0
# Compute diffusion pseudotime
sc.tl.dpt(adata2)
adata2

AnnData object with n_obs × n_vars = 13 × 974
    obs: 'cluster', 'dpt_pseudotime'
    var: 'xroot'
    uns: 'neighbors', 'diffmap_evals', 'iroot'
    obsm: 'X_diffmap'
    obsp: 'distances', 'connectivities'

In [28]:
adata1.obs

,cluster,dpt_pseudotime
0,12,0.000000
1,8,0.484134
2,1_CD8T,0.351462
3,7,0.593160
4,4_CD8T,0.445189
5,5_CD8Tex,0.448726
6,11,0.669826
7,0_CD8T,0.413974
8,10,0.776318
9,3_CD4Tconv,0.545269


In [29]:
adata2.obs

,cluster,dpt_pseudotime
0,12,0.000000
1,8,0.440135
2,6_CD8T,0.104250
3,7,0.906648
4,1_CD8T,0.845788
5,3_CD8T,0.850794
6,11,0.477240
7,9,0.282752
8,0_CD8T,0.197373
9,4_CD4Tconv,0.258326


In [33]:
adata1.obs[adata1.obs['cluster']==route1[i][j]]['dpt_pseudotime'].tolist()

[0.9882808327674866]

In [35]:
atc = []
for i in range(len(route1)):
    pt1 = []
    pt2 = []
    for j in range(len(route1[i])):
        pt1.append(adata1.obs[adata1.obs['cluster']==route1[i][j]]['dpt_pseudotime'].tolist()[0])
        pt2.append(adata2.obs[adata2.obs['cluster']==route2[i][j]]['dpt_pseudotime'].tolist()[0])
    atc.append(pd.Series.corr(pd.Series(pt1),pd.Series(pt2)))
    print(pt1,pt2)
    print(pd.Series.corr(pd.Series(pt1),pd.Series(pt2)))
atc = np.array(atc)

[0.0, 0.4841344356536865, 0.3514615297317505] [0.0, 0.44013532996177673, 0.1042499989271164]
0.8524153984872863
[0.0, 0.4841344356536865, 0.5931600332260132, 0.44518929719924927] [0.0, 0.44013532996177673, 0.9066480994224548, 0.8457878828048706]
0.8892901026949482
[0.0, 0.4841344356536865, 0.5931600332260132, 0.44872599840164185] [0.0, 0.44013532996177673, 0.9066480994224548, 0.8507938981056213]
0.8899761130844883
[0.0, 0.6698257923126221, 0.41397374868392944] [0.0, 0.4772396385669708, 0.19737288355827332]
0.972531515904382
[0.0, 0.6698257923126221] [0.0, 0.4772396385669708]
1.0
[0.0, 0.6698257923126221, 1.0, 0.9851980805397034] [0.0, 0.4772396385669708, 1.0, 0.9177668690681458]
0.9810000560113723
[0.0, 0.6698257923126221, 1.0, 0.9882808327674866] [0.0, 0.4772396385669708, 1.0, 0.9361177086830139]
0.9810329668098544


In [34]:
atc.mean()

0.9380351647131902

In [81]:
((atc + 1 )/2).mean()

0.9690175823565952

In [43]:
cell_route1= [ ]
for i in route1:
    t1 = []
    for j in i:
        if(len(j.split('_')) !=1):
            t1.append(j);
    if(len(t1)!=0):
        cell_route1.append(t1);
cell_route1

[['1_CD8T'],
 ['4_CD8T'],
 ['5_CD8Tex'],
 ['0_CD8T'],
 ['2_CD4Tconv'],
 ['6_CD4Tconv']]

In [44]:
cell_route2= [ ]
for i in route2:
    t1 = []
    for j in i:
        if(len(j.split('_')) !=1):
            t1.append(j);
    if(len(t1)!=0):
        cell_route2.append(t1);
cell_route2

[['6_CD8T'],
 ['1_CD8T'],
 ['3_CD8T'],
 ['0_CD8T'],
 ['2_CD4Tconv'],
 ['5_CD4Tconv']]

In [48]:
for i,j in zip(cell_route1,cell_route2):
    print(i,j)

['1_CD8T'] ['6_CD8T']
['4_CD8T'] ['1_CD8T']
['5_CD8Tex'] ['3_CD8T']
['0_CD8T'] ['0_CD8T']
['2_CD4Tconv'] ['2_CD4Tconv']
['6_CD4Tconv'] ['5_CD4Tconv']


In [38]:
route1,route2

([['12', '8', '1_CD8T'],
  ['12', '8', '7', '4_CD8T'],
  ['12', '8', '7', '5_CD8Tex'],
  ['12', '11', '0_CD8T'],
  ['12', '11'],
  ['12', '11', '9', '2_CD4Tconv'],
  ['12', '11', '9', '6_CD4Tconv']],
 [['12', '8', '6_CD8T'],
  ['12', '8', '7', '1_CD8T'],
  ['12', '8', '7', '3_CD8T'],
  ['12', '11', '0_CD8T'],
  ['12', '11'],
  ['12', '11', '10', '2_CD4Tconv'],
  ['12', '11', '10', '5_CD4Tconv']])

In [159]:
data1 = pd.read_csv('./datas/120/1/meta_result.csv')
data1 = data1.set_index(data1.columns[0])
gene1 = pd.read_csv('./datas/120/1/merge_values.csv')
gene1 = gene1.set_index(gene1.columns[0])
merged_df1 = pd.merge(data1['second'], gene1, left_index=True, right_index=True)
merged_df1 = merged_df1.drop('second',axis=1)
merged_df1 = merged_df1.drop('label',axis=1)
merged_df1.index = merged_df1.index.astype(str)

adata1 = sc.AnnData(merged_df1,dtype=np.float32)
# sc.tl.pca(
#     adata1,
#     n_comps=1000,
#     random_state=1234,
# )
sc.pp.neighbors(adata1,use_rep='X')
sc.tl.diffmap(adata1)


In [160]:
adata1.var['xroot'] = adata1.X.mean(axis=0)
# adata1.uns['iroot'] = 0
# Compute diffusion pseudotime
sc.tl.dpt(adata1)
adata1

AnnData object with n_obs × n_vars = 5000 × 16701
    obs: 'dpt_pseudotime'
    var: 'xroot'
    uns: 'neighbors', 'diffmap_evals', 'iroot'
    obsm: 'X_diffmap'
    obsp: 'distances', 'connectivities'

In [161]:
adata1.obs

,dpt_pseudotime
Unnamed: 0,
0,0.378134
1,0.405823
2,0.362443
3,0.175145
4,0.000000
...,...
4995,0.283985
4996,0.252008
4997,0.257369


In [4]:
data2 = pd.read_csv('./datas/120/2/meta_result.csv')
data2 = data2.set_index(data2.columns[0])
gene2 = pd.read_csv('./datas/120/2/merge_values.csv')
gene2 = gene2.set_index(gene2.columns[0])
merged_df2 = pd.merge(data2['second'], gene2, left_index=True, right_index=True)
merged_df2 = merged_df2.drop('second',axis=1)
merged_df2 = merged_df2.drop('label',axis=1)
merged_df2.index = merged_df2.index.astype(str)

adata2 = sc.AnnData(merged_df2,dtype=np.float32)
# sc.tl.pca(
#     adata2,
#     n_comps=1000,
#     random_state=1234,
# )
sc.pp.neighbors(adata2,use_rep='X')
sc.tl.diffmap(adata2)


In [6]:
adata2.X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
adata2.X.mean(axis=0)

array([0.0007342072, 0.017018905 , 0.004596726 , ..., 0.          ,
       0.0003981525, 0.0001986793], dtype=float32)

In [162]:
adata2.var['xroot'] = adata2.X.mean(axis=0)

# adata2.uns['iroot'] = 0

# Compute diffusion pseudotime
sc.tl.dpt(adata2)
adata2

AnnData object with n_obs × n_vars = 5000 × 16701
    obs: 'dpt_pseudotime'
    var: 'xroot'
    uns: 'neighbors', 'diffmap_evals', 'iroot'
    obsm: 'X_diffmap'
    obsp: 'distances', 'connectivities'

In [163]:
adata1.obs

,dpt_pseudotime
Unnamed: 0,
0,0.378134
1,0.405823
2,0.362443
3,0.175145
4,0.000000
...,...
4995,0.283985
4996,0.252008
4997,0.257369


In [164]:
adata2.obs

,dpt_pseudotime
Unnamed: 0,
0,0.403861
1,0.246074
2,0.048426
3,0.065390
4,0.107219
...,...
4995,0.130869
4996,0.129924
4997,0.145103


In [165]:
adata1.obs['second'] = data1['second'].tolist()
adata2.obs['second'] = data2['second'].tolist()


In [187]:
atc1 = []
atc2 = []
for i,j in anslist:

    pt1 = adata1.obs[adata1.obs['second']==i]['dpt_pseudotime']
    atc1.append(pt1.mean())
    pt2 = adata2.obs[adata2.obs['second']==i]['dpt_pseudotime']
    atc2.append(pt2.mean())
(pd.Series.corr(pd.Series(atc1),pd.Series(atc2)) +1) /2

0.8207450087293368

In [168]:
atc1,atc2

([0.3513510227203369,
  0.405419260263443,
  0.2694925367832184,
  0.1640537977218628,
  0.1475425511598587,
  0.16928182542324066],
 [0.12122153490781784,
  0.19314150512218475,
  0.1360488086938858,
  0.15845853090286255,
  0.08551795035600662,
  0.08991791307926178])

In [186]:
nodes1

[12,
 8,
 1_CD8T,
 7,
 4_CD8T,
 5_CD8Tex,
 11,
 0_CD8T,
 10,
 3_CD4Tconv,
 9,
 2_CD4Tconv,
 6_CD4Tconv]